# Load Data từ Bronze Layer sang Silver Layer

Notebook này sẽ đọc dữ liệu từ Bronze layer (MinIO) và xử lý để load vào các bảng Iceberg trong Silver layer với Nessie catalog.

## 1. Import Libraries và Khởi tạo Spark Session

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
import os

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Load_Bronze_To_Silver")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    # ===== Cấu hình MinIO (S3-compatible) =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")
print("✅ Spark Session đã được khởi tạo với Nessie catalog!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 08:39:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/04 08:39:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark Session đã được khởi tạo với Nessie catalog!


## 2. Load Dữ Liệu School từ Bronze Layer

In [2]:
# Đường dẫn đến các file CSV trong Bronze layer trên MinIO (2021-2025)
bronze_school_files = [
    "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_2021.csv",
    "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_2022.csv",
    "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_2023.csv",
    "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_2024.csv",
    "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_2025.csv"
]

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu School từ Bronze layer (2021-2025)...")

dataframes = []
for file_path in bronze_school_files:
    try:
        year = file_path.split("_")[-1].replace(".csv", "")
        print(f"  ⏳ Đang đọc file năm {year}...")
        
        df_temp = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(file_path)
        
        # Chỉ giữ 3 cột: TenTruong, MaTruong, TinhThanh
        df_temp = df_temp.select(
            col("TenTruong"),
            col("MaTruong"),
            col("TinhThanh")
        )
        
        dataframes.append(df_temp)
        print(f"  ✅ Đọc được {df_temp.count()} dòng từ năm {year}")
        
    except Exception as e:
        print(f"  ❌ Lỗi khi đọc file {file_path}: {str(e)}")

# Ghép tất cả các DataFrame lại với nhau
if dataframes:
    print("\n🔗 Đang ghép tất cả các file lại...")
    df_school_bronze = dataframes[0]
    for df in dataframes[1:]:
        df_school_bronze = df_school_bronze.union(df)
    
    print(f"✅ Tổng số dòng sau khi ghép: {df_school_bronze.count()}")
    
    # Lọc unique theo 3 cột
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_school_bronze = df_school_bronze.dropDuplicates(["TenTruong", "MaTruong", "TinhThanh"])
    
    print(f"✅ Số dòng sau khi lọc unique: {df_school_bronze.count()}")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_school_bronze.printSchema()
    print("\n🔍 Xem 10 dòng đầu tiên:")
    df_school_bronze.show(10, truncate=False)
else:
    print("❌ Không đọc được dữ liệu từ bất kỳ file nào!")

📖 Đang đọc dữ liệu School từ Bronze layer (2021-2025)...
  ⏳ Đang đọc file năm 2021...


25/11/04 08:39:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

  ✅ Đọc được 248 dòng từ năm 2021
  ⏳ Đang đọc file năm 2022...
  ✅ Đọc được 204 dòng từ năm 2022
  ⏳ Đang đọc file năm 2023...
  ✅ Đọc được 245 dòng từ năm 2023
  ⏳ Đang đọc file năm 2024...
  ✅ Đọc được 234 dòng từ năm 2024
  ⏳ Đang đọc file năm 2025...
  ✅ Đọc được 62 dòng từ năm 2025

🔗 Đang ghép tất cả các file lại...
✅ Tổng số dòng sau khi ghép: 993

🔄 Đang lọc dữ liệu unique...
✅ Số dòng sau khi lọc unique: 270

📊 Schema của dữ liệu Bronze:
root
 |-- TenTruong: string (nullable = true)
 |-- MaTruong: string (nullable = true)
 |-- TinhThanh: string (nullable = true)


🔍 Xem 10 dòng đầu tiên:
+-------------------------------------------+--------+-----------+
|TenTruong                                  |MaTruong|TinhThanh  |
+-------------------------------------------+--------+-----------+
|Đại học Ngoại Ngữ - Đại học Huế            |DHF     |Huế        |
|Đại học Việt Bắc                           |DVB     |Thái Nguyên|
|Đại học Công Nghệ và Quản Lý Hữu Nghị      |DCQ     |Hà Nội

## 3. Xử Lý và Transform Dữ Liệu School

In [3]:
# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu School...")

# Thêm timestamp cho created_at và updated_at
current_ts = current_timestamp()

df_school_silver = df_school_bronze.select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("TenTruong").cast("string").alias("schoolName"),
    col("TinhThanh").cast("string").alias("province"),
    current_ts.alias("created_at"),
    current_ts.alias("updated_at")
)

# Làm sạch dữ liệu: loại bỏ các dòng có giá trị null ở các cột quan trọng
df_school_silver = df_school_silver.filter(
    col("schoolId").isNotNull() & 
    col("schoolName").isNotNull()
)

print(f"✅ Đã xử lý xong {df_school_silver.count()} dòng dữ liệu")
print("\n📊 Schema của dữ liệu Silver:")
df_school_silver.printSchema()
print("\n🔍 Xem 10 dòng sau khi xử lý:")
df_school_silver.show(10, truncate=False)

🔄 Đang xử lý dữ liệu School...
✅ Đã xử lý xong 270 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- schoolId: string (nullable = true)
 |-- schoolName: string (nullable = true)
 |-- province: string (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem 10 dòng sau khi xử lý:
+--------+-------------------------------------------+-----------+--------------------------+--------------------------+
|schoolId|schoolName                                 |province   |created_at                |updated_at                |
+--------+-------------------------------------------+-----------+--------------------------+--------------------------+
|DHF     |Đại học Ngoại Ngữ - Đại học Huế            |Huế        |2025-11-04 08:39:36.768678|2025-11-04 08:39:36.768678|
|DVB     |Đại học Việt Bắc                           |Thái Nguyên|2025-11-04 08:39:36.768678|2025-11-04 08:39:36.768678|
|DCQ     |Đại học Công Nghệ và Quản Lý Hữu Nghị    

## 4. Load Dữ Liệu vào Bảng School trong Silver Layer

In [4]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.school...")

try:
    # Ghi dữ liệu vào bảng Iceberg với mode append hoặc overwrite
    df_school_silver.writeTo("nessie.silver_tables.school") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng school!")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.school...


25/11/04 08:39:39 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to silver_tables/school_2205b7f0-2539-499b-825b-0ad3b3b9d0e4/data/00000-57-0cd2066c-060c-4f04-8663-71d972aa7a5c-0-00001.parquet. This is unsupported
                                                                                

✅ Đã ghi dữ liệu thành công vào bảng school!


## 5. Kiểm Tra Dữ Liệu Đã Load vào Silver Layer

In [5]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.school...")

try:
    df_verify = spark.table("nessie.silver_tables.school")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify.printSchema()
    
    print("\n🔍 10 dòng đầu tiên:")
    df_verify.show(10, truncate=False)
    
    print("\n📈 Thống kê theo tỉnh/thành phố:")
    df_verify.groupBy("province").count().orderBy(desc("count")).show(10, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.school...

📊 Tổng số dòng trong bảng: 270

🔍 Schema của bảng:
root
 |-- schoolId: string (nullable = true)
 |-- schoolName: string (nullable = true)
 |-- province: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 10 dòng đầu tiên:
+--------+-------------------------------------------+-----------+--------------------------+--------------------------+
|schoolId|schoolName                                 |province   |created_at                |updated_at                |
+--------+-------------------------------------------+-----------+--------------------------+--------------------------+
|DHF     |Đại học Ngoại Ngữ - Đại học Huế            |Huế        |2025-11-04 08:39:38.408709|2025-11-04 08:39:38.408709|
|DVB     |Đại học Việt Bắc                           |Thái Nguyên|2025-11-04 08:39:38.408709|2025-11-04 08:39:38.408709|
|DCQ     |Đại học Công Nghệ và Quản Lý Hữu 

---

# LOAD DỮ LIỆU BẢNG MAJOR (NGÀNH HỌC)

---

## 6. Load Dữ Liệu Major từ Bronze Layer

In [6]:
# Đường dẫn đến file CSV trong Bronze layer trên MinIO
bronze_major_path = "s3a://bronze/structured_data/danh sách các nhóm ngành đại học/Danh_sách_các_ngành_theo_nhóm_ngành.csv"

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu Major từ Bronze layer...")

try:
    df_major_bronze = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_major_path)
    
    print(f"✅ Đã đọc được {df_major_bronze.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_major_bronze.printSchema()
    print("\n🔍 Xem 10 dòng đầu tiên:")
    df_major_bronze.show(10, truncate=False)
    
    # Xem các cột có trong file
    print("\n📋 Các cột trong file:")
    print(df_major_bronze.columns)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu Major từ Bronze layer...
✅ Đã đọc được 1791 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- MaNganh: string (nullable = true)
 |-- TenNganh: string (nullable = true)
 |-- MajorGroupId: integer (nullable = true)


🔍 Xem 10 dòng đầu tiên:
+-------+-------------------------------------------------------------------------------------------------------+------------+
|MaNganh|TenNganh                                                                                               |MajorGroupId|
+-------+-------------------------------------------------------------------------------------------------------+------------+
|7340302|KIỂM TOÁN (7340302)                                                                                    |1           |
|7340302|Kiểm toán (7340302)                                                                                    |1           |
|TM09   |Marketing (Marketing số) (TM09)                                                                

## 7. Xử Lý và Transform Dữ Liệu Major

In [10]:
# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Major...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # Giả sử file có các cột: MaNganh, TenNganh, MaNhomNganh
    # Điều chỉnh tên cột theo file thực tế của bạn
    df_major_silver = df_major_bronze.select(
        col(df_major_bronze.columns[0]).cast("string").alias("majorId"),          # Cột đầu tiên: Mã ngành
        col(df_major_bronze.columns[1]).cast("string").alias("majorName"),        # Cột thứ 2: Tên ngành
        col(df_major_bronze.columns[2]).cast("int").alias("majorGroupId"),        # Cột thứ 3: Mã nhóm ngành
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Làm sạch dữ liệu: loại bỏ các dòng có giá trị null ở các cột quan trọng
    df_major_silver = df_major_silver.filter(
        col("majorId").isNotNull() & 
        col("majorName").isNotNull() &
        col("majorGroupId").isNotNull()
    )
    
    # Lọc unique theo majorId và majorName
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_major_silver = df_major_silver.dropDuplicates(["majorId", "majorName"])
    
    print(f"✅ Đã xử lý xong {df_major_silver.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_major_silver.printSchema()
    print("\n🔍 Xem 10 dòng sau khi xử lý:")
    df_major_silver.show(10, truncate=False)
    
    # Thống kê theo nhóm ngành
    print("\n📈 Thống kê theo nhóm ngành:")
    df_major_silver.groupBy("majorGroupId").count().orderBy("majorGroupId").show(truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Major...

🔄 Đang lọc dữ liệu unique...
✅ Đã xử lý xong 733 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- majorId: string (nullable = true)
 |-- majorName: string (nullable = true)
 |-- majorGroupId: integer (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem 10 dòng sau khi xử lý:
+--------+-----------------------------------------------------------+------------+--------------------------+--------------------------+
|majorId |majorName                                                  |majorGroupId|created_at                |updated_at                |
+--------+-----------------------------------------------------------+------------+--------------------------+--------------------------+
|7340403 |Quản lý công (7340403)                                     |15          |2025-11-04 08:41:03.320811|2025-11-04 08:41:03.320811|
|EE      |Hệ thống điện và năng lượng tái tạo (CT tiên tiến) (EE-E18)|26

## 8. Load Dữ Liệu vào Bảng Major trong Silver Layer

In [11]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.major...")

try:
    # Ghi dữ liệu vào bảng Iceberg với mode append hoặc overwrite
    df_major_silver.writeTo("nessie.silver_tables.major") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng major!")
    print(f"📊 Tổng số dòng đã ghi: {df_major_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.major...
✅ Đã ghi dữ liệu thành công vào bảng major!
📊 Tổng số dòng đã ghi: 733


## 9. Kiểm Tra Dữ Liệu Major Đã Load vào Silver Layer

In [12]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.major...")

try:
    df_verify_major = spark.table("nessie.silver_tables.major")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_major.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_major.printSchema()
    
    print("\n🔍 10 dòng đầu tiên:")
    df_verify_major.show(10, truncate=False)
    
    print("\n📈 Thống kê theo nhóm ngành:")
    df_verify_major.groupBy("majorGroupId").count().orderBy("majorGroupId").show(20, truncate=False)
    
    print("\n🔍 Sample một vài ngành:")
    df_verify_major.select("majorId", "majorName", "majorGroupId").show(20, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.major...

📊 Tổng số dòng trong bảng: 733

🔍 Schema của bảng:
root
 |-- majorId: string (nullable = true)
 |-- majorName: string (nullable = true)
 |-- majorGroupId: integer (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 10 dòng đầu tiên:
+--------+-----------------------------------------------------------+------------+--------------------------+--------------------------+
|majorId |majorName                                                  |majorGroupId|created_at                |updated_at                |
+--------+-----------------------------------------------------------+------------+--------------------------+--------------------------+
|7340403 |Quản lý công (7340403)                                     |15          |2025-11-04 08:41:14.470901|2025-11-04 08:41:14.470901|
|EE      |Hệ thống điện và năng lượng tái tạo (CT tiên tiến) (EE-E18)|26          |2025-11-04 

---

# LOAD DỮ LIỆU BẢNG MAJOR_GROUP (NHÓM NGÀNH)

---

## 10. Load Dữ Liệu Major Group từ Bronze Layer

In [13]:
# Đường dẫn đến file CSV trong Bronze layer trên MinIO
bronze_major_group_path = "s3a://bronze/structured_data/danh sách các nhóm ngành đại học/Danh_sách_các_nhóm_ngành_đại_học.csv"

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu Major Group từ Bronze layer...")

try:
    df_major_group_bronze = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_major_group_path)
    
    print(f"✅ Đã đọc được {df_major_group_bronze.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_major_group_bronze.printSchema()
    print("\n🔍 Xem tất cả dữ liệu:")
    df_major_group_bronze.show(100, truncate=False)
    
    # Xem các cột có trong file
    print("\n📋 Các cột trong file:")
    print(df_major_group_bronze.columns)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu Major Group từ Bronze layer...
✅ Đã đọc được 34 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- STT: integer (nullable = true)
 |-- NhomNganh: string (nullable = true)
 |-- MajorGroupId: integer (nullable = true)
 |-- Link: string (nullable = true)


🔍 Xem tất cả dữ liệu:
+---+--------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------+
|STT|NhomNganh                                   |MajorGroupId|Link                                                                                                                              |
+---+--------------------------------------------+------------+----------------------------------------------------------------------------------------------------------------------------------+
|1  |Kế toán - Kiểm toán                         |1           |https://vietnamnet.vn/giao-duc/diem-thi/tra-cuu-diem-ch

## 11. Xử Lý và Transform Dữ Liệu Major Group

In [14]:
# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Major Group...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # Giả sử file có các cột: MaNhomNganh, TenNhomNganh
    # Điều chỉnh tên cột theo file thực tế của bạn
    df_major_group_silver = df_major_group_bronze.select(
        col(df_major_group_bronze.columns[2]).cast("int").alias("majorGroupId"),      # Cột đầu tiên: Mã nhóm ngành
        col(df_major_group_bronze.columns[1]).cast("string").alias("majorGroupName"), # Cột thứ 2: Tên nhóm ngành
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Làm sạch dữ liệu: loại bỏ các dòng có giá trị null ở các cột quan trọng
    df_major_group_silver = df_major_group_silver.filter(
        col("majorGroupId").isNotNull() & 
        col("majorGroupName").isNotNull()
    )
    
    # Lọc unique theo majorGroupId
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_major_group_silver = df_major_group_silver.dropDuplicates(["majorGroupId"])
    
    print(f"✅ Đã xử lý xong {df_major_group_silver.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_major_group_silver.printSchema()
    print("\n🔍 Xem tất cả dữ liệu sau khi xử lý:")
    df_major_group_silver.show(100, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Major Group...

🔄 Đang lọc dữ liệu unique...
✅ Đã xử lý xong 34 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- majorGroupId: integer (nullable = true)
 |-- majorGroupName: string (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem tất cả dữ liệu sau khi xử lý:
+------------+--------------------------------------------+--------------------------+--------------------------+
|majorGroupId|majorGroupName                              |created_at                |updated_at                |
+------------+--------------------------------------------+--------------------------+--------------------------+
|1           |Kế toán - Kiểm toán                         |2025-11-04 08:49:53.096761|2025-11-04 08:49:53.096761|
|2           |Tài chính - Ngân hàng - Bảo hiểm            |2025-11-04 08:49:53.096761|2025-11-04 08:49:53.096761|
|3           |Kinh tế - Quản trị kinh doanh - Thương Mại  |2025-11-04 08:49:

## 12. Load Dữ Liệu vào Bảng Major Group trong Silver Layer

In [15]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.major_group...")

try:
    df_major_group_silver.writeTo("nessie.silver_tables.major_group") \
    .using("iceberg") \
    .createOrReplace()

    print(" Đã ghi dữ liệu thành công vào bảng major_group!")
    print(f" Tổng số dòng đã ghi: {df_major_group_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.major_group...
 Đã ghi dữ liệu thành công vào bảng major_group!
 Tổng số dòng đã ghi: 34


## 13. Kiểm Tra Dữ Liệu Major Group Đã Load vào Silver Layer

In [16]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.major_group...")

try:
    df_verify_major_group = spark.table("nessie.silver_tables.major_group")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_major_group.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_major_group.printSchema()
    
    print("\n🔍 Tất cả nhóm ngành:")
    df_verify_major_group.orderBy("majorGroupId").show(100, truncate=False)
    
    # Kiểm tra xem có nhóm ngành nào bị thiếu không
    print("\n📊 Số lượng nhóm ngành:")
    print(f"Total: {df_verify_major_group.count()} nhóm ngành")
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.major_group...

📊 Tổng số dòng trong bảng: 34

🔍 Schema của bảng:
root
 |-- majorGroupId: integer (nullable = true)
 |-- majorGroupName: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 Tất cả nhóm ngành:
+------------+--------------------------------------------+--------------------------+--------------------------+
|majorGroupId|majorGroupName                              |created_at                |updated_at                |
+------------+--------------------------------------------+--------------------------+--------------------------+
|1           |Kế toán - Kiểm toán                         |2025-11-04 08:50:04.324695|2025-11-04 08:50:04.324695|
|2           |Tài chính - Ngân hàng - Bảo hiểm            |2025-11-04 08:50:04.324695|2025-11-04 08:50:04.324695|
|3           |Kinh tế - Quản trị kinh doanh - Thương Mại  |2025-11-04 08:50:04.324695|2025-11-04 08:50:0

---

# LOAD DỮ LIỆU BẢNG SUBJECT_GROUP (NHÓM MÔN/KHỐI THI)

---

## 14. Load Dữ Liệu Subject Group từ Bronze Layer

In [27]:
# Đường dẫn đến file CSV trong Bronze layer trên MinIO
bronze_subject_group_path = "s3a://bronze/structured_data/tohop_mon.csv"

print("📖 Đang đọc dữ liệu Subject Group từ Bronze layer...")

try:
    df_subject_group_bronze = (
        spark.read
        .format("csv")
        .option("header", "true")           # Có dòng tiêu đề
        .option("inferSchema", "true")      # Tự động suy luận kiểu dữ liệu
        .option("encoding", "UTF-8")        # Đọc tiếng Việt
        .csv(bronze_subject_group_path)
    )


    print(f"✅ Đã đọc được {df_subject_group_bronze.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_subject_group_bronze.printSchema()

    print("\n🔍 Xem dữ liệu mẫu:")
    df_subject_group_bronze.show(truncate=False)

    print("\n📋 Các cột trong file:")
    print(df_subject_group_bronze.columns)

except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()


📖 Đang đọc dữ liệu Subject Group từ Bronze layer...
✅ Đã đọc được 212 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- STT: integer (nullable = true)
 |-- Tổ hợp: string (nullable = true)
 |-- Môn chi tiết: string (nullable = true)


🔍 Xem dữ liệu mẫu:
+---+------+-------------------------+
|STT|Tổ hợp|Môn chi tiết             |
+---+------+-------------------------+
|1  |D01   |Toán-Ngữ Văn-Tiếng Anh   |
|2  |A00   |Toán-Vật lí-Hóa học      |
|3  |A01   |Toán-Vật lí-Tiếng Anh    |
|4  |D07   |Toán-Hóa học-Tiếng Anh   |
|5  |B00   |Toán-Hóa học-Sinh học    |
|6  |C00   |Ngữ văn-Lịch sử-Địa lí   |
|7  |C01   |Ngữ văn-Toán-Vật lí      |
|8  |D14   |Ngữ văn-Lịch sử-Tiếng Anh|
|9  |D15   |Ngữ văn-Địa lí-Tiếng Anh |
|10 |A02   |Toán-Vật lí-Sinh học     |
|11 |C04   |Ngữ văn-Toán-Địa lí      |
|12 |D03   |Ngữ văn-Toán-Tiếng Pháp  |
|13 |D04   |Ngữ văn-Toán-Tiếng Trung |
|14 |D10   |Toán-Địa lí-Tiếng Anh    |
|15 |B08   |Toán-Sinh học-Tiếng Anh  |
|16 |D06   |Ngữ văn-Toán-Tiếng Nhật  |
|1

In [30]:
# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Subject Group...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # File có 3 cột: STT, Tổ hợp, Môn chi tiết
    # Column 0: STT (subjectGroupId)
    # Column 1: Tổ hợp (subjectGroupName) - VD: "D01"
    # Column 2: Môn chi tiết (subjectCombination) - VD: "Toán, Ngữ Văn, Tiếng Anh"
    
    df_subject_group_silver = df_subject_group_bronze.select(
        col(df_subject_group_bronze.columns[0]).cast("int").alias("subjectGroupId"),           # STT
        col(df_subject_group_bronze.columns[1]).cast("string").alias("subjectGroupName"),      # Tổ hợp (VD: D01, A00, A01)
        col(df_subject_group_bronze.columns[2]).cast("string").alias("subjectCombination"),    # Môn chi tiết
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Làm sạch dữ liệu: loại bỏ các dòng có giá trị null ở các cột quan trọng
    df_subject_group_silver = df_subject_group_silver.filter(
        col("subjectGroupId").isNotNull() & 
        col("subjectGroupName").isNotNull() &
        col("subjectCombination").isNotNull()
    )
    
    # Lọc unique theo subjectGroupName
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_subject_group_silver = df_subject_group_silver.dropDuplicates(["subjectGroupName","subjectCombination"])
    
    print(f"✅ Đã xử lý xong {df_subject_group_silver.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_subject_group_silver.printSchema()
    print("\n🔍 Xem 20 dòng đầu tiên sau khi xử lý:")
    df_subject_group_silver.show(20, truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số tổ hợp môn: {df_subject_group_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Subject Group...

🔄 Đang lọc dữ liệu unique...
✅ Đã xử lý xong 211 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- subjectGroupId: integer (nullable = true)
 |-- subjectGroupName: string (nullable = true)
 |-- subjectCombination: string (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem 20 dòng đầu tiên sau khi xử lý:
+--------------+-------------------+-------------------------------------------------------------+-------------------------+-------------------------+
|subjectGroupId|subjectGroupName   |subjectCombination                                           |created_at               |updated_at               |
+--------------+-------------------+-------------------------------------------------------------+-------------------------+-------------------------+
|160           |AH2                |Toán-Hóa học-Tiếng Hàn                                       |2025-11-04 09:48:58.25058|2025-11-04

## 16. Load Dữ Liệu vào Bảng Subject Group trong Silver Layer

In [31]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.subject_group...")

try:
    df_subject_group_silver.writeTo("nessie.silver_tables.subject_group") \
    .using("iceberg") \
    .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng subject_group!")
    print(f"📊 Tổng số dòng đã ghi: {df_subject_group_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.subject_group...
✅ Đã ghi dữ liệu thành công vào bảng subject_group!
📊 Tổng số dòng đã ghi: 211


## 17. Kiểm Tra Dữ Liệu Subject Group Đã Load vào Silver Layer

In [32]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.subject_group...")

try:
    df_verify_subject_group = spark.table("nessie.silver_tables.subject_group")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_subject_group.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_subject_group.printSchema()
    
    print("\n🔍 20 tổ hợp môn đầu tiên:")
    df_verify_subject_group.orderBy("subjectGroupId").show(20, truncate=False)
    
    print("\n🔍 Một số tổ hợp môn cụ thể:")
    df_verify_subject_group.filter(col("subjectGroupName").isin(["A00", "A01", "D01", "C00"])).show(truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số tổ hợp môn: {df_verify_subject_group.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.subject_group...

📊 Tổng số dòng trong bảng: 211

🔍 Schema của bảng:
root
 |-- subjectGroupId: integer (nullable = true)
 |-- subjectGroupName: string (nullable = true)
 |-- subjectCombination: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 20 tổ hợp môn đầu tiên:
+--------------+----------------+-------------------------+--------------------------+--------------------------+
|subjectGroupId|subjectGroupName|subjectCombination       |created_at                |updated_at                |
+--------------+----------------+-------------------------+--------------------------+--------------------------+
|1             |D01             |Toán-Ngữ Văn-Tiếng Anh   |2025-11-04 09:49:05.887576|2025-11-04 09:49:05.887576|
|2             |A00             |Toán-Vật lí-Hóa học      |2025-11-04 09:49:05.887576|2025-11-04 09:49:05.887576|
|3             |A01             |Toán-Vật 

## 18. Load Dữ Liệu Selection Method từ Bronze Layer

In [33]:
# Đường dẫn đến file CSV điểm chuẩn trong Bronze layer trên MinIO
# File này chứa thông tin về phương thức xét tuyển trong cột PhuongThuc
bronze_benchmark_path = "s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv"

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu điểm chuẩn để lấy Selection Method từ Bronze layer...")

try:
    df_benchmark_bronze = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_benchmark_path)
    
    print(f"✅ Đã đọc được {df_benchmark_bronze.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_benchmark_bronze.printSchema()
    
    # Xem các giá trị trong cột PhuongThuc
    print("\n🔍 Các giá trị trong cột PhuongThuc:")
    df_benchmark_bronze.select("PhuongThuc").distinct().show(50, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu điểm chuẩn để lấy Selection Method từ Bronze layer...


✅ Đã đọc được 187555 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- STT: integer (nullable = true)
 |-- Nam: integer (nullable = true)
 |-- MaTruong: string (nullable = true)
 |-- PhuongThuc: string (nullable = true)
 |-- MaNganh: string (nullable = true)
 |-- TenNganh: string (nullable = true)
 |-- KhoiThi: string (nullable = true)
 |-- DiemChuan: double (nullable = true)
 |-- GhiChu: string (nullable = true)


🔍 Các giá trị trong cột PhuongThuc:
+----------------------------------------------------------------+
|PhuongThuc                                                      |
+----------------------------------------------------------------+
|Điểm chuẩn theo phương thức Điểm học bạ năm 2022                |
|Điểm chuẩn theo phương thức Điểm ĐGNL HN năm 2022               |
|Điểm chuẩn theo phương thức Điểm ĐGNL HN năm 2021               |
|Điểm chuẩn theo phương thức Điểm ĐGNL HCM năm 2022              |
|Điểm chuẩn theo phương thức Điểm ĐGNL HCM năm 2021              |
|Điểm 

## 19. Xử Lý và Transform Dữ Liệu Selection Method

In [34]:
from pyspark.sql.functions import regexp_replace, trim, row_number
from pyspark.sql.window import Window

# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Selection Method...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # Lấy cột PhuongThuc và loại bỏ phần "năm ..." 
    # VD: "Xét tuyển theo điểm thi THPT năm 2023" -> "Xét tuyển theo điểm thi THPT"
    df_selection_method = df_benchmark_bronze.select(
        regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "").alias("selectionMethodName_raw")
    ).distinct()
    
    # Loại bỏ khoảng trắng thừa và filter null
    df_selection_method = df_selection_method.withColumn(
        "selectionMethodName",
        trim(col("selectionMethodName_raw"))
    ).filter(
        col("selectionMethodName").isNotNull() & 
        (col("selectionMethodName") != "")
    ).select("selectionMethodName").distinct()
    
    # Tạo selectionMethodId tự động bằng row_number
    window_spec = Window.orderBy("selectionMethodName")
    df_selection_method_silver = df_selection_method.withColumn(
        "selectionMethodId",
        row_number().over(window_spec)
    ).select(
        col("selectionMethodId").cast("int"),
        col("selectionMethodName").cast("string"),
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    print(f"✅ Đã xử lý xong {df_selection_method_silver.count()} phương thức xét tuyển")
    print("\n📊 Schema của dữ liệu Silver:")
    df_selection_method_silver.printSchema()
    print("\n🔍 Tất cả các phương thức xét tuyển sau khi xử lý:")
    df_selection_method_silver.orderBy("selectionMethodId").show(100, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Selection Method...


25/11/04 09:55:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:55:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:55:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


✅ Đã xử lý xong 13 phương thức xét tuyển

📊 Schema của dữ liệu Silver:
root
 |-- selectionMethodId: integer (nullable = false)
 |-- selectionMethodName: string (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Tất cả các phương thức xét tuyển sau khi xử lý:
+-----------------+-------------------------------------------------------+--------------------------+--------------------------+
|selectionMethodId|selectionMethodName                                    |created_at                |updated_at                |
+-----------------+-------------------------------------------------------+--------------------------+--------------------------+
|1                |Điểm chuẩn theo phương thức Chứng chỉ quốc tế          |2025-11-04 09:55:55.476181|2025-11-04 09:55:55.476181|
|2                |Điểm chuẩn theo phương thức Điểm học bạ                |2025-11-04 09:55:55.476181|2025-11-04 09:55:55.476181|
|3                |Điểm chuẩ

25/11/04 09:55:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:55:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:55:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:55:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


## 20. Load Dữ Liệu vào Bảng Selection Method trong Silver Layer

In [35]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.selection_method...")

try:
    df_selection_method_silver.writeTo("nessie.silver_tables.selection_method") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng selection_method!")
    print(f"📊 Tổng số dòng đã ghi: {df_selection_method_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.selection_method...


25/11/04 09:56:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:56:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:56:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:56:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:56:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 09:56:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/04 0

✅ Đã ghi dữ liệu thành công vào bảng selection_method!
📊 Tổng số dòng đã ghi: 13


## 21. Kiểm Tra Dữ Liệu Selection Method Đã Load vào Silver Layer

In [36]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.selection_method...")

try:
    df_verify_selection_method = spark.table("nessie.silver_tables.selection_method")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_selection_method.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_selection_method.printSchema()
    
    print("\n🔍 Tất cả các phương thức xét tuyển:")
    df_verify_selection_method.orderBy("selectionMethodId").show(100, truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số phương thức xét tuyển: {df_verify_selection_method.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.selection_method...

📊 Tổng số dòng trong bảng: 13

🔍 Schema của bảng:
root
 |-- selectionMethodId: integer (nullable = true)
 |-- selectionMethodName: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 Tất cả các phương thức xét tuyển:
+-----------------+-------------------------------------------------------+--------------------------+--------------------------+
|selectionMethodId|selectionMethodName                                    |created_at                |updated_at                |
+-----------------+-------------------------------------------------------+--------------------------+--------------------------+
|1                |Điểm chuẩn theo phương thức Chứng chỉ quốc tế          |2025-11-04 09:56:39.355397|2025-11-04 09:56:39.355397|
|2                |Điểm chuẩn theo phương thức Điểm học bạ                |2025-11-04 09:56:39.355397|2025-11-04 09:56:39.35

---

# LOAD DỮ LIỆU BẢNG BENCHMARK (ĐIỂM CHUẨN)

---

## 22. Load Dữ Liệu Benchmark từ Bronze Layer

In [37]:
# Đường dẫn đến file CSV điểm chuẩn trong Bronze layer trên MinIO
bronze_benchmark_path = "s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv"

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu Benchmark từ Bronze layer...")

try:
    df_benchmark_raw = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_benchmark_path)
    
    print(f"✅ Đã đọc được {df_benchmark_raw.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_benchmark_raw.printSchema()
    
    print("\n📋 Các cột trong file:")
    print(df_benchmark_raw.columns)
    
    print("\n🔍 Xem 10 dòng đầu tiên:")
    df_benchmark_raw.show(10, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu Benchmark từ Bronze layer...
✅ Đã đọc được 187555 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- STT: integer (nullable = true)
 |-- Nam: integer (nullable = true)
 |-- MaTruong: string (nullable = true)
 |-- PhuongThuc: string (nullable = true)
 |-- MaNganh: string (nullable = true)
 |-- TenNganh: string (nullable = true)
 |-- KhoiThi: string (nullable = true)
 |-- DiemChuan: double (nullable = true)
 |-- GhiChu: string (nullable = true)


📋 Các cột trong file:
['STT', 'Nam', 'MaTruong', 'PhuongThuc', 'MaNganh', 'TenNganh', 'KhoiThi', 'DiemChuan', 'GhiChu']

🔍 Xem 10 dòng đầu tiên:
+---+----+--------+--------------------------------------------------+-------+-----------------+-------+---------+------------------------------+
|STT|Nam |MaTruong|PhuongThuc                                        |MaNganh|TenNganh         |KhoiThi|DiemChuan|GhiChu                        |
+---+----+--------+--------------------------------------------------+-------+---------------

## 23. Xử Lý và Transform Dữ Liệu Benchmark

In [38]:
from pyspark.sql.functions import regexp_replace, trim, monotonically_increasing_id, lit

# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Benchmark...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # Các cột trong file CSV:
    # STT, Nam, MaTruong, PhuongThuc, MaNganh, TenNganh, KhoiThi, DiemChuan, GhiChu
    
    # Bước 1: Xử lý PhuongThuc - loại bỏ phần "năm..."
    df_benchmark_processed = df_benchmark_raw.withColumn(
        "PhuongThuc_cleaned",
        trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", ""))
    )
    
    # Bước 2: Join với bảng selection_method để lấy selectionMethodId
    df_selection_method_lookup = spark.table("nessie.silver_tables.selection_method")
    
    df_benchmark_with_method = df_benchmark_processed.join(
        df_selection_method_lookup,
        df_benchmark_processed["PhuongThuc_cleaned"] == df_selection_method_lookup["selectionMethodName"],
        "left"
    )
    
    # Bước 3: Lấy subjectGroupId từ bảng subject_group dựa trên KhoiThi
    df_subject_group_lookup = spark.table("nessie.silver_tables.subject_group")
    
    df_benchmark_with_subject = df_benchmark_with_method.join(
        df_subject_group_lookup,
        df_benchmark_with_method["KhoiThi"] == df_subject_group_lookup["subjectGroupName"],
        "left"
    )
    
    # Bước 4: Select và cast các cột theo schema Silver
    df_benchmark_silver = df_benchmark_with_subject.select(
        col("MaTruong").cast("string").alias("schoolId"),
        col("MaNganh").cast("string").alias("majorId"),
        col("subjectGroupId").cast("int"),
        col("selectionMethodId").cast("int"),
        col("Nam").cast("int").alias("year"),
        col("DiemChuan").cast("double").alias("score"),
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Bước 5: Làm sạch dữ liệu - loại bỏ các dòng có giá trị null ở các cột quan trọng
    df_benchmark_silver = df_benchmark_silver.filter(
        col("schoolId").isNotNull() & 
        col("majorId").isNotNull() &
        col("year").isNotNull() &
        col("score").isNotNull() &
        col("selectionMethodId").isNotNull() &
        col("subjectGroupId").isNotNull()
    )
    
    # Bước 6: Thêm benchmarkId tự động tăng
    df_benchmark_silver = df_benchmark_silver.withColumn(
        "benchmarkId",
        monotonically_increasing_id().cast("int")
    ).select(
        "benchmarkId",
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "year",
        "score",
        "created_at",
        "updated_at"
    )
    
    # Bước 7: Lọc unique để tránh trùng lặp
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_benchmark_silver = df_benchmark_silver.dropDuplicates([
        "schoolId", "majorId", "subjectGroupId", "selectionMethodId", "year"
    ])
    
    print(f"✅ Đã xử lý xong {df_benchmark_silver.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_benchmark_silver.printSchema()
    
    print("\n🔍 Xem 20 dòng đầu tiên sau khi xử lý:")
    df_benchmark_silver.orderBy("year", "schoolId").show(20, truncate=False)
    
    # Thống kê theo năm
    print("\n📊 Thống kê theo năm:")
    df_benchmark_silver.groupBy("year").count().orderBy("year").show()
    
    # Thống kê điểm chuẩn
    print("\n📊 Thống kê điểm chuẩn:")
    df_benchmark_silver.select(
        avg("score").alias("Điểm TB"),
        min("score").alias("Điểm Min"),
        max("score").alias("Điểm Max")
    ).show()
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Benchmark...

🔄 Đang lọc dữ liệu unique...


✅ Đã xử lý xong 142855 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- benchmarkId: integer (nullable = false)
 |-- schoolId: string (nullable = true)
 |-- majorId: string (nullable = true)
 |-- subjectGroupId: integer (nullable = true)
 |-- selectionMethodId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem 20 dòng đầu tiên sau khi xử lý:


+-----------+--------+---------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|benchmarkId|schoolId|majorId  |subjectGroupId|selectionMethodId|year|score|created_at                |updated_at                |
+-----------+--------+---------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|8          |ANH     |7720101  |5             |3                |2021|24.45|2025-11-04 10:06:46.320926|2025-11-04 10:06:46.320926|
|17         |ANH     |7860100  |3             |3                |2021|27.24|2025-11-04 10:06:46.320926|2025-11-04 10:06:46.320926|
|4          |ANH     |7480202  |3             |3                |2021|26.55|2025-11-04 10:06:46.320926|2025-11-04 10:06:46.320926|
|10         |ANH     |7860100  |2             |3                |2021|26.2 |2025-11-04 10:06:46.320926|2025-11-04 10:06:46.320926|
|24         |ANH     |7860100  |40            |3                |2021|26.66|2025-11

[Stage 330:========>                                                (1 + 6) / 7]

+-----------------+--------+--------+
|          Điểm TB|Điểm Min|Điểm Max|
+-----------------+--------+--------+
|44.53931488992336|     0.0|  2600.0|
+-----------------+--------+--------+



## 24. Load Dữ Liệu vào Bảng Benchmark trong Silver Layer

In [39]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.benchmark...")

try:
    # Ghi dữ liệu vào bảng Iceberg với partitioning theo năm
    df_benchmark_silver.writeTo("nessie.silver_tables.benchmark") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng benchmark!")
    print(f"📊 Tổng số dòng đã ghi: {df_benchmark_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.benchmark...


✅ Đã ghi dữ liệu thành công vào bảng benchmark!
📊 Tổng số dòng đã ghi: 142855


## 25. Kiểm Tra Dữ Liệu Benchmark Đã Load vào Silver Layer

In [40]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.benchmark...")

try:
    df_verify_benchmark = spark.table("nessie.silver_tables.benchmark")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_benchmark.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_benchmark.printSchema()
    
    print("\n🔍 20 dòng đầu tiên:")
    df_verify_benchmark.orderBy("year", "score").show(20, truncate=False)
    
    print("\n📈 Thống kê theo năm:")
    df_verify_benchmark.groupBy("year") \
        .agg(
            count("*").alias("Số lượng"),
            avg("score").alias("Điểm TB"),
            min("score").alias("Điểm Min"),
            max("score").alias("Điểm Max")
        ) \
        .orderBy("year") \
        .show()
    
    print("\n📈 Top 10 điểm chuẩn cao nhất năm 2025:")
    df_verify_benchmark.filter(col("year") == 2025) \
        .orderBy(desc("score")) \
        .show(10, truncate=False)
    
    print("\n📊 Phân bố theo phương thức xét tuyển:")
    df_verify_benchmark.groupBy("selectionMethodId").count() \
        .orderBy(desc("count")) \
        .show(10)
    
    print("\n📊 Phân bố theo khối thi:")
    df_verify_benchmark.filter(col("subjectGroupId").isNotNull()) \
        .groupBy("subjectGroupId").count() \
        .orderBy(desc("count")) \
        .show(10)
    
    # Kiểm tra dữ liệu null
    print("\n⚠️ Kiểm tra dữ liệu null/missing:")
    print(f"Số dòng có subjectGroupId = null: {df_verify_benchmark.filter(col('subjectGroupId').isNull()).count()}")
    print(f"Số dòng có selectionMethodId = null: {df_verify_benchmark.filter(col('selectionMethodId').isNull()).count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.benchmark...

📊 Tổng số dòng trong bảng: 142855

🔍 Schema của bảng:
root
 |-- benchmarkId: integer (nullable = true)
 |-- schoolId: string (nullable = true)
 |-- majorId: string (nullable = true)
 |-- subjectGroupId: integer (nullable = true)
 |-- selectionMethodId: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- score: double (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 20 dòng đầu tiên:


+-----------+--------+-------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|benchmarkId|schoolId|majorId|subjectGroupId|selectionMethodId|year|score|created_at                |updated_at                |
+-----------+--------+-------+--------------+-----------------+----+-----+--------------------------+--------------------------+
|6296       |DQU     |7140202|1             |2                |2021|0.0  |2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|19632      |TDV     |7620115|1             |3                |2021|0.0  |2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|6313       |DQU     |7140213|5             |2                |2021|0.0  |2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|2333       |DDS     |7140246|2             |3                |2021|0.0  |2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|6295       |DQU     |7140202|6             |2                |2021|0.0  |2025-11-04 10:07:09.295

+-----------+--------+-------+--------------+-----------------+----+------+--------------------------+--------------------------+
|benchmarkId|schoolId|majorId|subjectGroupId|selectionMethodId|year|score |created_at                |updated_at                |
+-----------+--------+-------+--------------+-----------------+----+------+--------------------------+--------------------------+
|18409      |PKA     |FBE8   |3             |2                |2025|2333.0|2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|18410      |PKA     |FBE8   |15            |2                |2025|2333.0|2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|18338      |PKA     |FBE2   |3             |2                |2025|2333.0|2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|18337      |PKA     |FBE2   |2             |2                |2025|2333.0|2025-11-04 10:07:09.295945|2025-11-04 10:07:09.295945|
|18341      |PKA     |FBE2   |4             |2                |2025|2333.0|2025-11-04 10:0

+-----------------+-----+
|selectionMethodId|count|
+-----------------+-----+
|                3|78898|
|                2|51622|
|                5| 4697|
|                7| 2960|
|               13| 1580|
|               12| 1428|
|                9|  592|
|                4|  340|
|               11|  325|
|                8|  217|
+-----------------+-----+
only showing top 10 rows


📊 Phân bố theo khối thi:
+--------------+-----+
|subjectGroupId|count|
+--------------+-----+
|             1|25384|
|             2|25045|
|             3|22548|
|             4| 9641|
|             5| 7327|
|             6| 6771|
|             7| 4721|
|             8| 3193|
|             9| 3017|
|            11| 2275|
+--------------+-----+
only showing top 10 rows


⚠️ Kiểm tra dữ liệu null/missing:
Số dòng có subjectGroupId = null: 0
Số dòng có selectionMethodId = null: 0


---

# LOAD DỮ LIỆU BẢNG REGION (KHU VỰC)

---

## 26. Load Dữ Liệu Region từ Bronze Layer

In [41]:
# Đường dẫn đến file CSV region trong Bronze layer trên MinIO
bronze_region_path = "s3a://bronze/structured_data/region.csv"

# Đọc dữ liệu từ Bronze layer
print("📖 Đang đọc dữ liệu Region từ Bronze layer...")

try:
    df_region_bronze = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_region_path)
    
    print(f"✅ Đã đọc được {df_region_bronze.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Bronze:")
    df_region_bronze.printSchema()
    
    print("\n📋 Các cột trong file:")
    print(df_region_bronze.columns)
    
    print("\n🔍 Xem tất cả dữ liệu:")
    df_region_bronze.show(100, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu Region từ Bronze layer...
✅ Đã đọc được 64 dòng dữ liệu

📊 Schema của dữ liệu Bronze:
root
 |-- Mã đơn vị: integer (nullable = true)
 |-- Tên đơn vị: string (nullable = true)


📋 Các cột trong file:
['Mã đơn vị', 'Tên đơn vị']

🔍 Xem tất cả dữ liệu:
+---------+------------------------------+
|Mã đơn vị|Tên đơn vị                    |
+---------+------------------------------+
|1        |Sở GDĐT Hà Nội                |
|2        |Sở GDĐT TP. Hồ Chí Minh       |
|3        |Sở GDĐT Hải Phòng             |
|4        |Sở GDĐT Đà Nẵng               |
|5        |Sở GDĐT Hà Giang              |
|6        |Sở GDĐT Cao Bằng              |
|7        |Sở GDĐT Lai Châu              |
|8        |Sở GDĐT Lào Cai               |
|9        |Sở GDĐT Tuyên Quang           |
|10       |Sở GDĐT Lạng Sơn              |
|11       |Sở GDĐT Bắc Kạn               |
|12       |Sở GDĐT Thái Nguyên           |
|13       |Sở GDĐT Yên Bái               |
|14       |Sở GDĐT Sơn La                |


## 27. Xử Lý và Transform Dữ Liệu Region

In [42]:
# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Region...")

try:
    # Thêm timestamp cho created_at và updated_at
    current_ts = current_timestamp()
    
    # File có các cột: regionId, regionName (hoặc tương tự)
    # Điều chỉnh tên cột theo file thực tế
    
    # Nếu file không có tên cột chuẩn, sử dụng index
    df_region_silver = df_region_bronze.select(
        col(df_region_bronze.columns[0]).cast("string").alias("regionId"),
        col(df_region_bronze.columns[1]).cast("string").alias("regionName"),
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Làm sạch dữ liệu: loại bỏ các dòng có giá trị null ở các cột quan trọng
    df_region_silver = df_region_silver.filter(
        col("regionId").isNotNull() & 
        col("regionName").isNotNull()
    )
    
    # Lọc unique theo regionId
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_region_silver = df_region_silver.dropDuplicates(["regionId"])
    
    print(f"✅ Đã xử lý xong {df_region_silver.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_region_silver.printSchema()
    
    print("\n🔍 Tất cả các khu vực sau khi xử lý:")
    df_region_silver.orderBy("regionId").show(100, truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số khu vực: {df_region_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Region...

🔄 Đang lọc dữ liệu unique...
✅ Đã xử lý xong 64 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- regionId: string (nullable = true)
 |-- regionName: string (nullable = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Tất cả các khu vực sau khi xử lý:
+--------+------------------------------+--------------------------+--------------------------+
|regionId|regionName                    |created_at                |updated_at                |
+--------+------------------------------+--------------------------+--------------------------+
|1       |Sở GDĐT Hà Nội                |2025-11-04 10:45:14.706721|2025-11-04 10:45:14.706721|
|10      |Sở GDĐT Lạng Sơn              |2025-11-04 10:45:14.706721|2025-11-04 10:45:14.706721|
|11      |Sở GDĐT Bắc Kạn               |2025-11-04 10:45:14.706721|2025-11-04 10:45:14.706721|
|12      |Sở GDĐT Thái Nguyên           |2025-11-04 10:45:14.706721|2025-11-04 10:45

## 28. Load Dữ Liệu vào Bảng Region trong Silver Layer

In [43]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.region...")

try:
    df_region_silver.writeTo("nessie.silver_tables.region") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng region!")
    print(f"📊 Tổng số dòng đã ghi: {df_region_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.region...
✅ Đã ghi dữ liệu thành công vào bảng region!
📊 Tổng số dòng đã ghi: 64


## 29. Kiểm Tra Dữ Liệu Region Đã Load vào Silver Layer

In [44]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.region...")

try:
    df_verify_region = spark.table("nessie.silver_tables.region")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_region.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_region.printSchema()
    
    print("\n🔍 Tất cả các khu vực:")
    df_verify_region.orderBy("regionId").show(100, truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số khu vực: {df_verify_region.count()}")
    
    # Lưu DataFrame để sử dụng cho việc map với student_scores sau này
    print("\n💾 Lưu thông tin region để sử dụng cho mapping...")
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.region...

📊 Tổng số dòng trong bảng: 64

🔍 Schema của bảng:
root
 |-- regionId: string (nullable = true)
 |-- regionName: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 Tất cả các khu vực:
+--------+------------------------------+--------------------------+--------------------------+
|regionId|regionName                    |created_at                |updated_at                |
+--------+------------------------------+--------------------------+--------------------------+
|1       |Sở GDĐT Hà Nội                |2025-11-04 10:45:27.614805|2025-11-04 10:45:27.614805|
|10      |Sở GDĐT Lạng Sơn              |2025-11-04 10:45:27.614805|2025-11-04 10:45:27.614805|
|11      |Sở GDĐT Bắc Kạn               |2025-11-04 10:45:27.614805|2025-11-04 10:45:27.614805|
|12      |Sở GDĐT Thái Nguyên           |2025-11-04 10:45:27.614805|2025-11-04 10:45:27.614805|
|13      |Sở GD

---

# LOAD DỮ LIỆU BẢNG STUDENT_SCORES (ĐIỂM THI SINH VIÊN)

---

## 30. Load Dữ Liệu Student Scores từ Bronze Layer

In [65]:
# Đường dẫn đến các thư mục chứa file CSV điểm thi từng năm
# Cấu trúc: bronze/structured_data/điểm từng thi sinh/[2021-2025]/Diem_thi_*.csv
years = [2021, 2022, 2023, 2024, 2025]

print("📖 Đang đọc dữ liệu Student Scores từ Bronze layer (2021-2025)...")

try:
    all_dataframes = []
    
    for year in years:
        bronze_path = f"s3a://bronze/structured_data/điểm từng thí sinh/{year}/*.csv"
        print(f"\n⏳ Đang đọc dữ liệu năm {year}...")
        
        try:
            # Đọc tất cả file CSV trong thư mục năm
            df_year = spark.read \
                .option("header", "true") \
                .option("inferSchema", "false") \
                .option("encoding", "UTF-8") \
                .csv(bronze_path)
            
            # Thêm cột năm để phân biệt
            df_year = df_year.withColumn("Year", lit(year))
            
            count = df_year.count()
            print(f"  ✅ Đọc được {count:,} dòng từ năm {year}")
            
            all_dataframes.append(df_year)
            
        except Exception as e:
            print(f"  ⚠️ Không tìm thấy hoặc lỗi khi đọc dữ liệu năm {year}: {str(e)}")
    
    # Ghép tất cả các DataFrame lại
    if all_dataframes:
        print("\n🔗 Đang ghép tất cả dữ liệu các năm...")
        df_student_scores_bronze = all_dataframes[0]
        for df in all_dataframes[1:]:
            df_student_scores_bronze = df_student_scores_bronze.union(df)
        
        total_count = df_student_scores_bronze.count()
        print(f"✅ Tổng số dòng sau khi ghép: {total_count:,}")
        
        print("\n📊 Schema của dữ liệu Bronze:")
        df_student_scores_bronze.printSchema()
        
        print("\n📋 Các cột trong file:")
        print(df_student_scores_bronze.columns)
        
        print("\n🔍 Xem 10 dòng đầu tiên:")
        df_student_scores_bronze.show(10, truncate=False)
        
    else:
        print("❌ Không đọc được dữ liệu từ bất kỳ năm nào!")
        
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

📖 Đang đọc dữ liệu Student Scores từ Bronze layer (2021-2025)...

⏳ Đang đọc dữ liệu năm 2021...
  ✅ Đọc được 974,139 dòng từ năm 2021

⏳ Đang đọc dữ liệu năm 2022...
  ✅ Đọc được 968,471 dòng từ năm 2022

⏳ Đang đọc dữ liệu năm 2023...
  ✅ Đọc được 1,025,333 dòng từ năm 2023

⏳ Đang đọc dữ liệu năm 2024...
  ✅ Đọc được 1,061,466 dòng từ năm 2024

⏳ Đang đọc dữ liệu năm 2025...
  ✅ Đọc được 1,152,914 dòng từ năm 2025

🔗 Đang ghép tất cả dữ liệu các năm...


✅ Tổng số dòng sau khi ghép: 5,182,323

📊 Schema của dữ liệu Bronze:
root
 |-- SBD: string (nullable = true)
 |-- Cum: string (nullable = true)
 |-- DiemThi: string (nullable = true)
 |-- Year: integer (nullable = false)


📋 Các cột trong file:
['SBD', 'Cum', 'DiemThi', 'Year']

🔍 Xem 10 dòng đầu tiên:
+--------+-----------------------+----------------------------------------------------------+----+
|SBD     |Cum                    |DiemThi                                                   |Year|
+--------+-----------------------+----------------------------------------------------------+----+
|16000001|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:8.6,Văn:9,Sử:8.5,Địa:8.75,Ngoại ngữ:10,GDCD:9.75     |2021|
|16000002|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:8.4,Văn:8.75,Sử:8.75,Địa:8.75,Ngoại ngữ:9.4,GDCD:9.75|2021|
|16000003|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:7.6,Văn:8.75,Sử:9.25,Địa:10,Ngoại ngữ:8.4,GDCD:10    |2021|
|16000004|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:6.4,Văn:8.25,Sử:9.75,Địa:9.5,Ngoại ngữ:6.6,GDCD:9.75 |2021|
|16

## 31. Xử Lý và Transform Dữ Liệu Student Scores

In [66]:
from pyspark.sql.functions import split, expr, create_map, concat, lpad
from pyspark.sql.types import MapType, StringType, DoubleType
from itertools import chain

# Transform dữ liệu để phù hợp với schema Silver
print("🔄 Đang xử lý dữ liệu Student Scores...")

try:
    # Thêm timestamp
    current_ts = current_timestamp()
    
    # Bước 1: Tạo studentId = SBD + Year (VD: 010000012021)
    df_processed = df_student_scores_bronze.withColumn(
        "studentId",
        concat(col("SBD"), col("Year").cast("string"))
    )
    print("\n🔍 Xem 2 dòng sau khi tạo studentId:")
    df_processed.show(2, truncate=False)
    
    # Bước 2: Xử lý cột DiemThi - chuyển từ string sang MAP
    # Format: "Toán:2.2,Văn:3.5,Sử:2.5,Địa:5.5"
    # Cần chuyển thành: {"Toán": 2.2, "Văn": 3.5, "Sử": 2.5, "Địa": 5.5}
    
    # Split string thành array các cặp "môn:điểm"
    df_processed = df_processed.withColumn(
        "score_pairs",
        split(col("DiemThi"), ",")
    )

    
    # UDF để parse điểm từ string sang MAP
    from pyspark.sql.functions import udf
    from typing import Dict
    
    def parse_scores(score_string: str) -> Dict[str, float]:
        """Parse score string to dictionary"""
        if not score_string or score_string.strip() == "":
            return {}
        
        scores_dict = {}
        try:
            # Split by comma
            pairs = score_string.split(",")
            for pair in pairs:
                if ":" in pair:
                    subject, score = pair.split(":")
                    subject = subject.strip()
                    try:
                        scores_dict[subject] = float(score.strip())
                    except ValueError:
                        # Nếu không parse được điểm, bỏ qua
                        pass
        except Exception:
            pass
        
        return scores_dict
    
    # Đăng ký UDF
    parse_scores_udf = udf(parse_scores, MapType(StringType(), DoubleType()))
    
    # Apply UDF để tạo MAP scores
    df_processed = df_processed.withColumn(
        "scores",
        parse_scores_udf(col("DiemThi"))
    )
    print("\n🔍 Xem 2 dòng sau khi xử lý cột scores:")
    df_processed.show(1, truncate=False)
    # Bước 3: Xử lý cột SBD để lấy regionId
    # 2 ký tự đầu tiên của SBD chính là mã khu vực (regionId)
    # VD: SBD = "01000001" -> regionId = "01"
    df_processed = df_processed.withColumn(
        "regionId",
        substring(col("SBD"), 1, 2).cast("string")
    )
    print("\n🔍 Xem 2 dòng sau khi xử lý cột regionId:")
    df_processed.show(2, truncate=False)
    # Bước 4: Select các cột theo schema Silver
    df_student_scores_silver = df_processed.select(
        col("studentId").cast("string"),
        col("regionId").cast("string"),
        col("Year").cast("int").alias("year"),
        col("scores"),
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    # Bước 5: Làm sạch dữ liệu
    df_student_scores_silver = df_student_scores_silver.filter(
        col("studentId").isNotNull() & 
        col("year").isNotNull() &
        col("scores").isNotNull()
    )
    
    # Bước 6: Lọc unique theo studentId
    print("\n🔄 Đang lọc dữ liệu unique...")
    df_student_scores_silver = df_student_scores_silver.dropDuplicates(["studentId"])
    
    print(f"✅ Đã xử lý xong {df_student_scores_silver.count():,} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu Silver:")
    df_student_scores_silver.printSchema()
    
    print("\n🔍 Xem 10 dòng đầu tiên sau khi xử lý:")
    df_student_scores_silver.show(10, truncate=False)
    
    # Thống kê theo năm
    print("\n📊 Thống kê theo năm:")
    df_student_scores_silver.groupBy("year").count().orderBy("year").show()
    
    # Thống kê về số môn thi
    print("\n📊 Thống kê số môn thi:")
    df_student_scores_silver.select(
        size(col("scores")).alias("num_subjects")
    ).groupBy("num_subjects").count().orderBy("num_subjects").show()
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔄 Đang xử lý dữ liệu Student Scores...

🔍 Xem 2 dòng sau khi tạo studentId:
+--------+-----------------------+----------------------------------------------------------+----+------------+
|SBD     |Cum                    |DiemThi                                                   |Year|studentId   |
+--------+-----------------------+----------------------------------------------------------+----+------------+
|16000001|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:8.6,Văn:9,Sử:8.5,Địa:8.75,Ngoại ngữ:10,GDCD:9.75     |2021|160000012021|
|16000002|Sở GD&ĐT Tỉnh Vĩnh Phúc|Toán:8.4,Văn:8.75,Sử:8.75,Địa:8.75,Ngoại ngữ:9.4,GDCD:9.75|2021|160000022021|
+--------+-----------------------+----------------------------------------------------------+----+------------+
only showing top 2 rows


🔍 Xem 2 dòng sau khi xử lý cột scores:
+--------+-----------------------+-----------------------------------------------------+----+------------+------------------------------------------------------------+--------------------

25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:43:10 WARN RowBasedKeyValueBatch: Calling spill() on

✅ Đã xử lý xong 5,170,351 dòng dữ liệu

📊 Schema của dữ liệu Silver:
root
 |-- studentId: string (nullable = true)
 |-- regionId: string (nullable = true)
 |-- year: integer (nullable = false)
 |-- scores: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- created_at: timestamp (nullable = false)
 |-- updated_at: timestamp (nullable = false)


🔍 Xem 10 dòng đầu tiên sau khi xử lý:


+------------+--------+----+----------------------------------------------------------------------------------+--------------------------+--------------------------+
|studentId   |regionId|year|scores                                                                            |created_at                |updated_at                |
+------------+--------+----+----------------------------------------------------------------------------------+--------------------------+--------------------------+
|010000022022|01      |2022|{Văn -> 6.75, Địa -> 7.5, Ngoại ngữ -> 7.6, Sử -> 8.5, GDCD -> 8.25, Toán -> 8.4} |2025-11-04 11:43:11.230365|2025-11-04 11:43:11.230365|
|010000072022|01      |2022|{Văn -> 6.0, Địa -> 7.5, Ngoại ngữ -> 5.0, Sử -> 8.0, GDCD -> 9.0, Toán -> 7.2}   |2025-11-04 11:43:11.230365|2025-11-04 11:43:11.230365|
|010000082025|01      |2025|{Văn -> 6.25, Địa -> 4.0, Sử -> 4.0}                                              |2025-11-04 11:43:11.230365|2025-11-04 11:43:11.230365|
|010

25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:44:02 WARN RowBasedKeyValueBatch: Calling spill() on

+----+-------+
|year|  count|
+----+-------+
|2021| 965591|
|2022| 968471|
|2023|1021909|
|2024|1061466|
|2025|1152914|
+----+-------+


📊 Thống kê số môn thi:


[Stage 728:==================================================>    (12 + 1) / 13]

+------------+-------+
|num_subjects|  count|
+------------+-------+
|           0|   4636|
|           1|   2733|
|           2|   3210|
|           3| 138975|
|           4|1513992|
|           5|  56659|
|           6|3450146|
+------------+-------+



## 32. Load Dữ Liệu vào Bảng Student Scores trong Silver Layer

In [67]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.student_scores...")

try:
    # Ghi dữ liệu vào bảng Iceberg với partitioning theo năm
    df_student_scores_silver.writeTo("nessie.silver_tables.student_scores") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng student_scores!")
    print(f"📊 Tổng số dòng đã ghi: {df_student_scores_silver.count():,}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.student_scores...


✅ Đã ghi dữ liệu thành công vào bảng student_scores!


25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:58 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/11/04 11:50:59 WARN RowBasedKeyValueBatch: Calling spill() on

📊 Tổng số dòng đã ghi: 5,170,351


In [68]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.student_scores...")

try:
    df_verify_student_scores = spark.table("nessie.silver_tables.student_scores")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_student_scores.count():,}")
    print("\n🔍 Schema của bảng:")
    df_verify_student_scores.printSchema()
    
    print("\n🔍 20 dòng đầu tiên:")
    df_verify_student_scores.orderBy("year", "studentId").show(20, truncate=False)
    
    # print("\n📈 Thống kê theo năm:")
    # df_verify_student_scores.groupBy("year") \
    #     .agg(
    #         count("*").alias("Số thí sinh")
    #     ) \
    #     .orderBy("year") \
    #     .show()
    
    # print("\n📊 Phân bố số môn thi:")
    # df_verify_student_scores.select(
    #     size(col("scores")).alias("num_subjects")
    # ).groupBy("num_subjects") \
    #     .count() \
    #     .orderBy("num_subjects") \
    #     .show()
    
    print("\n🔍 Xem chi tiết điểm một vài thí sinh:")
    df_verify_student_scores.select(
        "studentId",
        "year",
        "scores"
    ).show(10, truncate=False)
    
    # # Thống kê về điểm số (ví dụ: môn Toán)
    # print("\n📊 Thống kê điểm môn Toán (nếu có):")
    # df_verify_student_scores.select(
    #     col("scores")["Toán"].alias("diem_toan")
    # ).filter(
    #     col("diem_toan").isNotNull()
    # ).select(
    #     avg("diem_toan").alias("Điểm TB Toán"),
    #     min("diem_toan").alias("Điểm Min Toán"),
    #     max("diem_toan").alias("Điểm Max Toán")
    # ).show()
    
    # # Kiểm tra dữ liệu null
    # print("\n⚠️ Kiểm tra dữ liệu null/missing:")
    # print(f"Số dòng có regionId = null: {df_verify_student_scores.filter(col('regionId').isNull()).count():,}")
    # print(f"Số dòng có scores rỗng: {df_verify_student_scores.filter(size(col('scores')) == 0).count():,}")
    
    # # Thống kê các môn thi phổ biến
    # print("\n📊 Các môn thi có trong dữ liệu (lấy mẫu):")
    # sample_df = df_verify_student_scores.limit(1000)
    # # Lấy tất cả keys từ MAP scores
    # all_keys = sample_df.select(explode(map_keys(col("scores")))).distinct()
    # print("Các môn thi:")
    # all_keys.show(50, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.student_scores...

📊 Tổng số dòng trong bảng: 5,170,351

🔍 Schema của bảng:
root
 |-- studentId: string (nullable = true)
 |-- regionId: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- scores: map (nullable = true)
 |    |-- key: string
 |    |-- value: double (valueContainsNull = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)


🔍 20 dòng đầu tiên:


[Stage 744:==============>                                          (1 + 3) / 4]

+------------+--------+----+------------------------------------------------------------------------------------+--------------------------+--------------------------+
|studentId   |regionId|year|scores                                                                              |created_at                |updated_at                |
+------------+--------+----+------------------------------------------------------------------------------------+--------------------------+--------------------------+
|010000012021|01      |2021|{Văn -> 3.5, Địa -> 5.5, Sử -> 2.5, Toán -> 2.2}                                    |2025-11-04 11:50:18.090262|2025-11-04 11:50:18.090262|
|010000022021|01      |2021|{Văn -> 8.25, Địa -> 9.75, Ngoại ngữ -> 10.0, Sử -> 5.75, GDCD -> 9.25, Toán -> 9.2}|2025-11-04 11:50:18.090262|2025-11-04 11:50:18.090262|
|010000032021|01      |2021|{Văn -> 6.25, Địa -> 4.0, Sử -> 4.25, Toán -> 4.4}                                  |2025-11-04 11:50:18.090262|2025-11-04 11:50:18.

---

# LOAD DỮ LIỆU BẢNG SUBJECT (MÔN HỌC)

---

## 34. Load và Tách Dữ Liệu Subject từ Bronze Layer

In [ ]:
# Đường dẫn đến file CSV trong Bronze layer
bronze_subject_group_path = "s3a://bronze/structured_data/tohop_mon.csv"

print("📖 Đang đọc dữ liệu Subject từ Bronze layer...")

try:
    # Đọc file tohop_mon.csv
    df_tohop_mon = spark.read \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("encoding", "UTF-8") \
        .csv(bronze_subject_group_path)
    
    print(f"✅ Đã đọc được {df_tohop_mon.count()} dòng dữ liệu")
    print("\n📊 Schema của dữ liệu:")
    df_tohop_mon.printSchema()
    
    print("\n📋 Các cột trong file:")
    print(df_tohop_mon.columns)
    
    print("\n🔍 Xem dữ liệu mẫu:")
    df_tohop_mon.show(10, truncate=False)
    
    # Lấy cột "Môn chi tiết" (cột thứ 3)
    mon_chi_tiet_col = df_tohop_mon.columns[2]
    print(f"\n📌 Tên cột môn chi tiết: {mon_chi_tiet_col}")
    
    # Lấy tất cả các giá trị môn chi tiết
    df_mon_chi_tiet = df_tohop_mon.select(col(mon_chi_tiet_col).alias("monChiTiet"))
    
    print("\n🔍 Các tổ hợp môn:")
    df_mon_chi_tiet.show(20, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

## 35. Xử Lý và Tách Môn Học từ Cột Môn Chi Tiết

In [ ]:
from pyspark.sql.functions import explode, split, trim, row_number
from pyspark.sql.window import Window

print("🔄 Đang xử lý và tách môn học...")

try:
    # Bước 1: Tách chuỗi môn chi tiết thành array
    # VD: "Toán-Ngữ Văn-Tiếng Anh" -> ["Toán", "Ngữ Văn", "Tiếng Anh"]
    df_mon_array = df_mon_chi_tiet.withColumn(
        "mon_array",
        split(col("monChiTiet"), "-")
    )
    
    print("\n🔍 Sau khi tách thành array:")
    df_mon_array.show(5, truncate=False)
    
    # Bước 2: Explode array thành các dòng riêng lẻ
    df_mon_exploded = df_mon_array.select(
        explode(col("mon_array")).alias("subjectName")
    )
    
    print(f"\n📊 Tổng số dòng sau khi explode: {df_mon_exploded.count()}")
    
    # Bước 3: Trim spaces và lấy unique
    df_mon_unique = df_mon_exploded.withColumn(
        "subjectName",
        trim(col("subjectName"))
    ).filter(
        col("subjectName").isNotNull() & 
        (col("subjectName") != "")
    ).distinct()
    
    print(f"\n📊 Số lượng môn học unique: {df_mon_unique.count()}")
    
    print("\n🔍 Danh sách các môn học:")
    df_mon_unique.orderBy("subjectName").show(50, truncate=False)
    
    # Bước 4: Thêm subjectId tự động
    current_ts = current_timestamp()
    window_spec = Window.orderBy("subjectName")
    
    df_subject_silver = df_mon_unique.withColumn(
        "subjectId",
        row_number().over(window_spec)
    ).select(
        col("subjectId").cast("int"),
        col("subjectName").cast("string"),
        current_ts.alias("created_at"),
        current_ts.alias("updated_at")
    )
    
    print(f"\n✅ Đã xử lý xong {df_subject_silver.count()} môn học")
    print("\n📊 Schema của dữ liệu Silver:")
    df_subject_silver.printSchema()
    
    print("\n🔍 Danh sách môn học với ID:")
    df_subject_silver.orderBy("subjectId").show(50, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi xử lý dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

## 36. Load Dữ Liệu vào Bảng Subject trong Silver Layer

In [ ]:
# Load dữ liệu vào bảng Iceberg trong Silver layer
print("💾 Đang ghi dữ liệu vào bảng nessie.silver_tables.subject...")

try:
    df_subject_silver.writeTo("nessie.silver_tables.subject") \
        .using("iceberg") \
        .createOrReplace()
    
    print("✅ Đã ghi dữ liệu thành công vào bảng subject!")
    print(f"📊 Tổng số dòng đã ghi: {df_subject_silver.count()}")
    
except Exception as e:
    print(f"❌ Lỗi khi ghi dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()

## 37. Kiểm Tra Dữ Liệu Subject Đã Load vào Silver Layer

In [ ]:
# Đọc và kiểm tra dữ liệu từ bảng Silver
print("🔍 Kiểm tra dữ liệu trong bảng nessie.silver_tables.subject...")

try:
    df_verify_subject = spark.table("nessie.silver_tables.subject")
    
    print(f"\n📊 Tổng số dòng trong bảng: {df_verify_subject.count()}")
    print("\n🔍 Schema của bảng:")
    df_verify_subject.printSchema()
    
    print("\n🔍 Tất cả các môn học:")
    df_verify_subject.orderBy("subjectId").show(50, truncate=False)
    
    # Thống kê
    print("\n📊 Thống kê:")
    print(f"Tổng số môn học: {df_verify_subject.count()}")
    
    # Liệt kê các môn học theo thứ tự alphabet
    print("\n📋 Danh sách môn học theo thứ tự alphabet:")
    df_verify_subject.orderBy("subjectName").show(50, truncate=False)
    
except Exception as e:
    print(f"❌ Lỗi khi đọc dữ liệu: {str(e)}")
    import traceback
    traceback.print_exc()